In [1]:
# imports
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
import json
import seaborn as sns
from rich import print
import time
from numba import njit
import seaborn as sns
from dask_ml.xgboost import XGBClassifier
sns.set_style('dark')

load_dotenv()


/home/netanel/.conda/envs/big_data_project/lib/python3.12/site-packages/dask_xgboost/__init__.py:7: UserWarning: Dask-XGBoost has been deprecated and is no longer maintained. The functionality of this project has been included directly in XGBoost. To use Dask and XGBoost together, please use ``xgboost.dask`` instead https://xgboost.readthedocs.io/en/latest/tutorials/dask.html.
  warnings.warn(


True

In [2]:
RAW_DATA_DIR = Path(os.getenv('RAW_DATA_DIR'))
PROCESSED_DATA_DIR = Path(os.getenv('PROCESSED_DATA_DIR'))
RANDOM_SEED = 42

In [8]:
views_filtered = pl.read_parquet(PROCESSED_DATA_DIR / 'views_filtered.parquet')
all_info = pl.read_parquet(PROCESSED_DATA_DIR / 'all_info.parquet')

In [9]:
all_info

wikidata_code,birth,death,gender,level1_main_occ,name,un_subregion,level2_main_occ,level3_main_occ,bigperiod_birth,bigperiod_death,sum_visib_ln_5criteria,ranking_visib_5criteria,citizenship_1_b,un_region,bplo1,dplo1,bpla1,dpla1,en_label,en_description,worked_at,educated_at,instrument,genre,birth_month,count_gender,count_instrument,count_worked_at,count_country_of_citizenship,count_educated_at,count_occupation,count_genre,is_one_hit_wonder,is_nostalgia,seasonality,is_seasonal,is_rising,is_fading,is_constant
str,i64,i64,str,str,str,str,str,str,str,str,f64,f64,str,str,f64,f64,f64,f64,str,str,list[str],list[str],list[str],list[str],str,u32,u32,u32,u32,u32,u32,u32,bool,bool,f64,bool,bool,bool,bool
"""Q7088052""",null,null,"""Female""","""Culture""","""Olivia_Giovett…","""Northern Ameri…","""Culture-core""","""author""","""Missing""","""Missing""",15.269195,2.255541e6,"""US""","""America""",null,null,null,null,"""Olivia Giovett…","""journalist""",[],[],[],[],"""01""",1,0,0,1,0,4,0,true,false,NaN,false,false,false,false
"""Q4719004""",1824,1862,"""Male""","""Other""","""Alexander_Gran…","""Oceania Wester…","""Worker/Busines…","""surveyor""","""4.Mid Modern P…","""4.Mid Modern P…",15.934114,1.747964e6,"""Australia""","""Oceania""",null,null,null,null,"""Alexander Gran…","""Surveyor""",[],[],[],[],"""01""",1,0,0,1,0,0,0,false,true,NaN,false,false,false,false
"""Q16067381""",1889,1980,"""Male""","""Leadership""","""Bernard_H._Rae…","""Northern Ameri…","""Politics""","""politician""","""4.Mid Modern P…","""5.Contemporary…",15.606245,1816228.5,"""US""","""America""",null,null,null,null,"""Bernard H. Rae…","""American polit…",[],"[""University of Wisconsin–Eau Claire""]",[],[],"""06""",1,0,0,1,1,1,0,false,false,9.0,true,false,false,false
"""Q16731813""",null,null,"""Female""","""Other""","""Leigh_Magar""",null,"""Worker/Busines…","""milliner""","""Missing""","""Missing""",15.060712,2388995.5,null,null,null,null,null,null,"""Leigh Magar""","""American milli…",[],"[""Fashion Institute of Technology""]",[],[],null,1,0,0,0,1,1,0,false,false,NaN,false,false,false,true
"""Q17486602""",1935,null,"""Male""","""Sports/Games""","""Boncho_Novakov…","""Eastern Europe…","""Sports/Games""","""sport""","""5.Contemporary…","""Missing""",16.396002,1.286861e6,"""Bulgaria""","""Europe""",26.528818,null,43.540894,null,"""Boncho Novakov…","""cyclist""",[],[],[],[],"""05""",1,0,0,1,0,1,0,false,false,NaN,false,false,false,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Q25735""",1973,null,"""Male""","""Sports/Games""","""Milorad_Mažić""","""Southern Europ…","""Sports/Games""","""football""","""5.Contemporary…","""Missing""",27.753937,240783.0,"""Serbia""","""Europe""",19.637777,null,45.569721,null,"""Milorad Mažić""","""Serbian intern…",[],[],[],[],"""03""",1,0,0,1,0,1,0,true,false,NaN,false,false,false,false
"""Q5361773""",1843,1918,"""Male""","""Leadership""","""Elisha_Jessop""","""Northern Ameri…","""Politics""","""politician""","""4.Mid Modern P…","""5.Contemporary…",14.515863,2.640646e6,"""Canada""","""America""",null,null,null,null,"""Elisha Jessop""","""Canadian polit…",[],[],[],[],"""01""",1,0,0,1,0,1,0,false,false,13.0,true,false,false,false
"""Q5237804""",1947,null,"""Male""","""Leadership""","""David_Mulready…","""Oceania Wester…","""Religious""","""priest""","""5.Contemporary…","""Missing""",15.84019,2.156183e6,"""Australia""","""Oceania""",151.210007,null,-33.867779,null,"""David Mulready…","""Australian Ang…",[],"[""Newington College""]",[],[],"""09""",1,0,0,0,1,1,0,false,false,NaN,false,false,false,true


In [148]:
feature_cols_cat = ['gender', 'level1_main_occ', 'un_subregion', 'level2_main_occ',
                'level3_main_occ', 'bigperiod_birth', 'citizenship_1_b', 'un_region',
                'birth_month']
feature_cols_all = feature_cols_cat + [c for c in all_info.columns if c.startswith('count_')] + ['birth', 'death']
label_cols = ['is_one_hit_wonder']
all_columns = feature_cols_all + label_cols

df = (all_info.select(all_columns)
      .with_columns([pl.col(c).str.replace_all(r' ', '_').str.replace_all(r'\.', '_').str.to_lowercase() for c in feature_cols_cat])
      .to_dummies(feature_cols_cat)
      .with_columns(pl.col('is_one_hit_wonder').cast(pl.UInt8))
)
df

gender_female,gender_male,gender_null,gender_other,level1_main_occ_culture,level1_main_occ_discovery/science,level1_main_occ_leadership,level1_main_occ_missing,level1_main_occ_other,level1_main_occ_sports/games,un_subregion_caribbean,un_subregion_central_africa,un_subregion_central_america,un_subregion_central_asia,un_subregion_east_africa,un_subregion_eastern_asia,un_subregion_eastern_europe,un_subregion_north_africa,un_subregion_northern_america,un_subregion_northern_europe,un_subregion_null,un_subregion_oceania_not_aus_nze,un_subregion_oceania_western_world,un_subregion_south_america,un_subregion_south_asia_incl__indian_peninsula,un_subregion_southeast_asia,un_subregion_southern_africa,un_subregion_southern_europe,un_subregion_west_africa,un_subregion_western_asia_(middle_east_caucasus),un_subregion_western_europe,level2_main_occ_academia,level2_main_occ_administration/law,level2_main_occ_corporate/executive/business_(large),level2_main_occ_culture-core,level2_main_occ_culture-periphery,level2_main_occ_explorer/inventor/developer,…,citizenship_1_b_vietnam,citizenship_1_b_wales,citizenship_1_b_western_sahara,citizenship_1_b_yemen,citizenship_1_b_yugoslavia,citizenship_1_b_zambia,citizenship_1_b_zanzibar,citizenship_1_b_zimbabwe,un_region_africa,un_region_america,un_region_asia,un_region_europe,un_region_null,un_region_oceania,birth_month_01,birth_month_02,birth_month_03,birth_month_04,birth_month_05,birth_month_06,birth_month_07,birth_month_08,birth_month_09,birth_month_10,birth_month_11,birth_month_12,birth_month_null,count_gender,count_instrument,count_worked_at,count_country_of_citizenship,count_educated_at,count_occupation,count_genre,birth,death,is_one_hit_wonder
u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u32,u32,u32,u32,u32,u32,u32,i64,i64,u8
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,…,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,4,0,null,null,1
0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1824,1862,0
0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,1,0,1889,1980,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,null,null,0
0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1935,null,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1973,null,1
0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1843,1918,0
0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,1947,null,0


In [134]:
[c for c in df.columns if '.' in c]

[]

In [149]:
df.write_parquet(PROCESSED_DATA_DIR / 'spark_df.parquet')

In [3]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.sql.functions import col, isnull
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassificationModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import pyspark
from pyspark.sql import SQLContext

In [4]:
sc = pyspark.SparkContext.getOrCreate()
sql_context = SQLContext(sc)

24/04/15 18:17:14 WARN Utils: Your hostname, gpucluster1 resolves to a loopback address: 127.0.1.1; using 132.72.107.155 instead (on interface enp106s0f0)
24/04/15 18:17:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/15 18:17:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/netanel/.conda/envs/big_data_project/lib/python3.12/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [7]:
sparkdf = sql_context.read.parquet(str(PROCESSED_DATA_DIR / 'spark_df.parquet'))
sparkdf = sparkdf.sample(0.25)
sparkdf.registerTempTable('df')

/home/netanel/.conda/envs/big_data_project/lib/python3.12/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [8]:
# Combine all features into one column
assembler = VectorAssembler(inputCols=sparkdf.columns, outputCol='features')
sparkdf = assembler.transform(sparkdf)

In [9]:
train, test = sparkdf.randomSplit((0.7, 0.3), seed=42)

In [12]:
# Create model
clf = DecisionTreeClassifier(labelCol='is_one_hit_wonder')
# Fit model
model = clf.fit(train.select(['features', 'is_one_hit_wonder']))
# Make predictions
predictions = model.predict(test.select(['features', 'is_one_hit_wonder']))
# evaluate predictions
evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
evaluation = evaluator.evaluate(predictions)

Exception in thread "RemoteBlock-temp-file-clean-thread" java.lang.OutOfMemoryError: GC overhead limit exceeded
24/04/15 18:18:57 ERROR Executor: Exception in task 58.0 in stage 3.0 (TID 61)
java.lang.OutOfMemoryError: GC overhead limit exceeded
24/04/15 18:18:57 ERROR Executor: Exception in task 6.0 in stage 3.0 (TID 9)
java.lang.IllegalStateException: unread block data
	at java.io.ObjectInputStream$BlockDataInputStream.setBlockDataMode(ObjectInputStream.java:2934)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1704)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2431)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2355)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2213)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1669)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:503)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:461)
	at org.apache.spark

Py4JError: An error occurred while calling o108.getParam

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/netanel/.conda/envs/big_data_project/lib/python3.12/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/netanel/.conda/envs/big_data_project/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/netanel/.conda/envs/big_data_project/lib/python3.12/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
